In [3]:
import numpy as np
import pandas as pd
import glob
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import load_model
import os
import gc
from pathlib import Path

In [23]:
path = Path('TORCS_DATA/data/')

In [30]:
track_files = list(path.glob("*/*.csv"))
# Picking the Wheel tracks for experiments
track_files = track_files[-2:]
track_files

[WindowsPath('TORCS_DATA/data/wheel-2/8.csv'),
 WindowsPath('TORCS_DATA/data/wheel-2/9.csv')]

In [34]:
df = pd.read_csv(track_files[0])
df

,Angle,CurrentLapTime,Damage,DistanceFromStart,DistanceCovered,FuelLevel,Gear,LastLapTime,Opponent_1,Opponent_2,...,WheelSpinVelocity_1,WheelSpinVelocity_2,WheelSpinVelocity_3,WheelSpinVelocity_4,Z,Acceleration,Braking,Clutch,Gear,Steering
0,0.000000,-0.982,0.0,6176.960,0.0,94.0000,0,0.000,17.0,200.0,...,0.0000,0.0000,0.0000,0.0000,0.345257,1.0,0.0,0.64,1,9.785958e-08
1,0.000000,-0.962,0.0,6176.960,0.0,94.0000,0,0.000,17.0,200.0,...,0.0000,0.0000,0.0000,0.0000,0.345257,1.0,0.0,0.64,1,9.785958e-08
2,0.000000,-0.942,0.0,6176.960,0.0,93.9999,0,0.000,17.0,200.0,...,0.0000,0.0000,0.0000,0.0000,0.345257,1.0,0.0,0.64,1,9.785958e-08
3,0.000000,-0.922,0.0,6176.960,0.0,93.9998,0,0.000,17.0,200.0,...,0.0000,0.0000,0.0000,0.0000,0.345257,1.0,0.0,0.64,1,9.785958e-08
4,0.000000,-0.902,0.0,6176.960,0.0,93.9997,0,0.000,17.0,200.0,...,0.0000,0.0000,0.0000,0.0000,0.345257,1.0,0.0,0.64,1,9.785958e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17756,-0.040338,10.256,88.0,555.351,19200.1,81.1824,2,113.972,200.0,200.0,...,89.6396,94.3795,92.7554,96.8785,0.340167,1.0,0.0,0.00,2,-3.091483e-01
17757,-0.036540,10.276,88.0,556.016,19200.8,81.1824,2,113.972,200.0,200.0,...,89.2155,94.0287,92.4108,96.5746,0.340302,1.0,0.0,0.00,2,-3.093583e-01
17758,-0.032683,10.296,88.0,556.679,19201.5,81.1824,2,113.972,200.0,200.0,...,88.8312,93.6578,92.0916,96.2478,0.340311,1.0,0.0,0.00,2,-3.095381e-01
17759,-0.028615,10.316,88.0,557.338,19202.1,81.1824,2,113.972,200.0,200.0,...,88.4758,93.2983,91.7693,95.9183,0.340465,1.0,0.0,0.00,2,-3.096908e-01


In [35]:
df.columns

Index(['Angle', ' CurrentLapTime', ' Damage', ' DistanceFromStart',
       ' DistanceCovered', ' FuelLevel', ' Gear', ' LastLapTime',
       ' Opponent_1', 'Opponent_2', 'Opponent_3', 'Opponent_4', 'Opponent_5',
       'Opponent_6', 'Opponent_7', 'Opponent_8', 'Opponent_9', 'Opponent_10',
       'Opponent_11', 'Opponent_12', 'Opponent_13', 'Opponent_14',
       'Opponent_15', 'Opponent_16', 'Opponent_17', 'Opponent_18',
       'Opponent_19', 'Opponent_20', 'Opponent_21', 'Opponent_22',
       'Opponent_23', 'Opponent_24', 'Opponent_25', 'Opponent_26',
       'Opponent_27', 'Opponent_28', 'Opponent_29', 'Opponent_30',
       'Opponent_31', 'Opponent_32', 'Opponent_33', 'Opponent_34',
       'Opponent_35', 'Opponent_36', 'RacePosition', ' RPM', ' SpeedX',
       ' SpeedY', ' SpeedZ', ' Track_1', 'Track_2', 'Track_3', 'Track_4',
       'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9', 'Track_10',
       'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Track_15', 'Track_16',
       'T

In [19]:
data = pd.DataFrame(columns=features)
for track in track_files:
    data = pd.concat([data, pd.read_csv(track, index_col=None)]).reset_index(drop=True)

In [20]:
data

,Angle,CurrentLapTime,Damage,DistanceFromStart,DistanceCovered,FuelLevel,Gear,LastLapTime,Opponent_1,Opponent_2,...,10.4433,8.2822,7.99999,0.333332,0.0.9,0.0.10,0.345263,Wheel2,0.0.11,-0.028559213504195213
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.99260,3.85590,3.68671,-0.385933,82.1721,83.1274,0.341080,wheel2,0.0,-0.058131
497535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.01076,3.87379,3.70555,-0.382754,82.1620,83.1165,0.341105,wheel2,0.0,-0.058181
497536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.02778,3.89084,3.72359,-0.379715,82.1488,83.1053,0.341105,wheel2,0.0,-0.058253
497537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.04365,3.90697,3.74078,-0.376816,82.1367,83.0934,0.341083,wheel2,0.0,-0.058350
